In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr, spearmanr
import numpy as np

c:\Users\User\SC4020 Project 1\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("data\huggingface-2025-10-14.csv")
df.head()

<>:1: SyntaxWarning: invalid escape sequence '\h'
<>:1: SyntaxWarning: invalid escape sequence '\h'
C:\Users\User\AppData\Local\Temp\ipykernel_38772\3814104705.py:1: SyntaxWarning: invalid escape sequence '\h'
  df = pd.read_csv("data\huggingface-2025-10-14.csv")
C:\Users\User\AppData\Local\Temp\ipykernel_38772\3814104705.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data\huggingface-2025-10-14.csv")


,score,sentence1,sentence2,score_binary
0,5.00,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,1.0
1,4.75,A young child is riding a horse.,A child is riding a horse.,1.0
2,5.00,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,1.0
3,2.40,A woman is playing the guitar.,A man is playing guitar.,0.0
4,2.75,A woman is playing the flute.,A man is playing a flute.,0.0


In [12]:
df = df[:1500]

In [3]:
def jaccard_similarity(sentence1, sentence2):
    set1 = set(sentence1.lower().split())
    set2 = set(sentence2.lower().split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

In [4]:
# --- Load model ---
model = SentenceTransformer('all-MiniLM-L6-v2')

In [13]:
# --- Prepare storage ---
cosine_scores = []
jaccard_scores = []

# --- Loop through pairs ---
total_pairs = len(df)
for i, (s1, s2) in enumerate(zip(df['sentence1'], df['sentence2']), start=1):
    # Encode & compute cosine similarity
    emb1 = model.encode([s1], normalize_embeddings=True)
    emb2 = model.encode([s2], normalize_embeddings=True)
    sim_cos = cosine_similarity(emb1, emb2)[0][0]
    cosine_scores.append(sim_cos)

    # Compute Jaccard similarity
    sim_jaccard = jaccard_similarity(s1, s2)
    jaccard_scores.append(sim_jaccard)

    # Print progress every 100 iterations or at the end
    if i % 100 == 0 or i == total_pairs:
        print(f"Processed {i}/{total_pairs} sentence pairs...")

# --- Store results in dataframe ---
df['cosine_score'] = cosine_scores
df['cosine_score_scaled'] = df['cosine_score'] * 5  # Scale to match gold score scale
df['jaccard_score'] = jaccard_scores
df['jaccard_score_scaled'] = df['jaccard_score'] * 5

# --- Prepare arrays ---
gold_scores = df['score'].values
pred_cosine = df['cosine_score_scaled'].values
pred_jaccard = df['jaccard_score_scaled'].values

# --- Compute correlations ---
pearson_cosine, _ = pearsonr(gold_scores, pred_cosine)
spearman_cosine, _ = spearmanr(gold_scores, pred_cosine)

pearson_jaccard, _ = pearsonr(gold_scores, pred_jaccard)
spearman_jaccard, _ = spearmanr(gold_scores, pred_jaccard)

# --- Print results ---
print("\n=== Correlation Results ===")
print(f"✅ Cosine  - Pearson: {pearson_cosine:.4f} | Spearman: {spearman_cosine:.4f}")
print(f"✅ Jaccard - Pearson: {pearson_jaccard:.4f} | Spearman: {spearman_jaccard:.4f}")

# --- Optional: quick inspection ---
print("\nSample predictions:")
print(df[['sentence1', 'sentence2', 'score', 'cosine_score_scaled', 'jaccard_score_scaled']].head())

Processed 100/1500 sentence pairs...
Processed 200/1500 sentence pairs...
Processed 300/1500 sentence pairs...
Processed 400/1500 sentence pairs...
Processed 500/1500 sentence pairs...
Processed 600/1500 sentence pairs...
Processed 700/1500 sentence pairs...
Processed 800/1500 sentence pairs...
Processed 900/1500 sentence pairs...
Processed 1000/1500 sentence pairs...
Processed 1100/1500 sentence pairs...
Processed 1200/1500 sentence pairs...
Processed 1300/1500 sentence pairs...
Processed 1400/1500 sentence pairs...
Processed 1500/1500 sentence pairs...

=== Correlation Results ===
✅ Cosine  - Pearson: 0.8696 | Spearman: 0.8672
✅ Jaccard - Pearson: 0.5960 | Spearman: 0.6017

Sample predictions:
                              sentence1  \
0     A man with a hard hat is dancing.   
1      A young child is riding a horse.   
2  A man is feeding a mouse to a snake.   
3        A woman is playing the guitar.   
4         A woman is playing the flute.   

                                  se

C:\Users\User\AppData\Local\Temp\ipykernel_38772\3383960630.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cosine_score'] = cosine_scores
C:\Users\User\AppData\Local\Temp\ipykernel_38772\3383960630.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cosine_score_scaled'] = df['cosine_score'] * 5  # Scale to match gold score scale
C:\Users\User\AppData\Local\Temp\ipykernel_38772\3383960630.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us